In [1]:
pip install gensim

     |████████████████████████████████| 23.9 MB 20.8 MB/s eta 0:00:01
     |████████████████████████████████| 56 kB 549 kB/s  eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import joblib, os
from gensim.models import Word2Vec
import pandas as pd
import datetime

import boto3
from sagemaker import get_execution_role

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [3]:
role = get_execution_role()
connection = boto3.client('s3')
contents = connection.list_objects(Bucket='ascsagemaker',
                                  Prefix="JMP_congressional_nmf/latest_dtms")['Contents']

chamber_csv = [f['Key'] for f in contents if f['Key'].endswith('.csv')]
chamber_csv

['JMP_congressional_nmf/latest_dtms/097.csv',
 'JMP_congressional_nmf/latest_dtms/098.csv',
 'JMP_congressional_nmf/latest_dtms/099.csv',
 'JMP_congressional_nmf/latest_dtms/100.csv',
 'JMP_congressional_nmf/latest_dtms/101.csv',
 'JMP_congressional_nmf/latest_dtms/102.csv',
 'JMP_congressional_nmf/latest_dtms/103.csv',
 'JMP_congressional_nmf/latest_dtms/104.csv',
 'JMP_congressional_nmf/latest_dtms/105.csv',
 'JMP_congressional_nmf/latest_dtms/106.csv',
 'JMP_congressional_nmf/latest_dtms/107.csv',
 'JMP_congressional_nmf/latest_dtms/108.csv',
 'JMP_congressional_nmf/latest_dtms/109.csv',
 'JMP_congressional_nmf/latest_dtms/110.csv',
 'JMP_congressional_nmf/latest_dtms/111.csv',
 'JMP_congressional_nmf/latest_dtms/112.csv',
 'JMP_congressional_nmf/latest_dtms/113.csv',
 'JMP_congressional_nmf/latest_dtms/114.csv']

In [5]:
all_speeches = []
for f in chamber_csv:
#     if f.endswith('102.csv'):
    speeches = pd.read_csv(connection.get_object(Bucket='ascsagemaker',Key=f)['Body'])['speech_processed']
#     speeches = [speech.split() for speech in speeches]
    all_speeches.extend(speeches)
    print(f)

JMP_congressional_nmf/latest_dtms/097.csv
JMP_congressional_nmf/latest_dtms/098.csv
JMP_congressional_nmf/latest_dtms/099.csv
JMP_congressional_nmf/latest_dtms/100.csv
JMP_congressional_nmf/latest_dtms/101.csv
JMP_congressional_nmf/latest_dtms/102.csv
JMP_congressional_nmf/latest_dtms/103.csv
JMP_congressional_nmf/latest_dtms/104.csv
JMP_congressional_nmf/latest_dtms/105.csv
JMP_congressional_nmf/latest_dtms/106.csv
JMP_congressional_nmf/latest_dtms/107.csv
JMP_congressional_nmf/latest_dtms/108.csv
JMP_congressional_nmf/latest_dtms/109.csv
JMP_congressional_nmf/latest_dtms/110.csv
JMP_congressional_nmf/latest_dtms/111.csv
JMP_congressional_nmf/latest_dtms/112.csv
JMP_congressional_nmf/latest_dtms/113.csv
JMP_congressional_nmf/latest_dtms/114.csv


In [7]:
for i,x in enumerate(['a','b']):
    print(i,x)

0 a
1 b


In [11]:
if 12 % 100 == 0:
    print('a')

In [12]:
def splitter(i,x):
    if i % 100 == 0:
        print("*",end='')
    return x.split()

all_speeches = [splitter(i,x) for i,x in enumerate(all_speeches)]

IndentationError: expected an indented block (<ipython-input-12-3c41feadbeee>, line 3)

In [2]:
model = Word2Vec(all_speeches,window=10,sg=1,workers=8)
print(f"\nmodel training time: {str(datetime.timedelta(seconds=model.total_train_time))}")

NameError: name 'Word2Vec' is not defined

In [7]:
model.save('word2vec-model_unigram.bin')

In [14]:
print(f"\nmodel training time: {str(datetime.timedelta(seconds=model.total_train_time))}")


model training time: 1:08:45.936644


In [47]:
connection.upload_file('word2vec-model.bin', 'ascsagemaker', 'JMP_congressional_nmf/word2vec-first20/word2vec-first20.bin')
connection.upload_file('word2vec-model.bin.syn1neg.npy', 'ascsagemaker', 'JMP_congressional_nmf/word2vec-first20/word2vec-model.bin.syn1neg.npy')
connection.upload_file('word2vec-model.bin.wv.vectors.npy', 'ascsagemaker', 'JMP_congressional_nmf/word2vec-first20/word2vec-model.bin.wv.vectors.npy')

In [44]:
w2v = gensim.models.Word2Vec.load('word2vec-model.bin')